In [24]:
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import re
from collections import Counter
from nltk import ngrams
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
df = pd.read_csv("20_newsgroup.csv")
df

,Unnamed: 0,text,target,title,date
0,0,I was wondering if anyone out there could enli...,7,rec.autos,2022-12-05 17:04:16.745380
1,17,I recently posted an article asking what kind ...,7,rec.autos,2022-12-05 17:04:16.745380
2,29,\nIt depends on your priorities. A lot of peo...,7,rec.autos,2022-12-05 17:04:16.745380
3,56,an excellent automatic can be found in the sub...,7,rec.autos,2022-12-05 17:04:16.745380
4,64,: Ford and his automobile. I need information...,7,rec.autos,2022-12-05 17:04:16.745380
...,...,...,...,...,...
11309,11210,Secrecy in Clipper Chip\n\nThe serial number o...,11,sci.crypt,2022-12-05 17:04:16.745380
11310,11217,Hi !\n\nI am interested in the source of FEAL ...,11,sci.crypt,2022-12-05 17:04:16.745380
11311,11243,"The actual algorithm is classified, however, t...",11,sci.crypt,2022-12-05 17:04:16.745380
11312,11254,\n\tThis appears to be generic calling upon th...,11,sci.crypt,2022-12-05 17:04:16.745380


In [17]:
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('date', axis=1)
df = df.drop('title', axis=1)
df.to_csv("use.csv", index = False)

In [10]:
df['target'].value_counts()
df['title'].value_counts()

rec.sport.hockey            600
soc.religion.christian      599
rec.motorcycles             598
rec.sport.baseball          597
sci.crypt                   595
rec.autos                   594
sci.med                     594
comp.windows.x              593
sci.space                   593
comp.os.ms-windows.misc     591
sci.electronics             591
comp.sys.ibm.pc.hardware    590
misc.forsale                585
comp.graphics               584
comp.sys.mac.hardware       578
talk.politics.mideast       564
talk.politics.guns          546
alt.atheism                 480
talk.politics.misc          465
talk.religion.misc          377
Name: title, dtype: int64

In [18]:
df = pd.read_csv("use.csv")
df

,text,target
0,I was wondering if anyone out there could enli...,7
1,I recently posted an article asking what kind ...,7
2,\nIt depends on your priorities. A lot of peo...,7
3,an excellent automatic can be found in the sub...,7
4,: Ford and his automobile. I need information...,7
...,...,...
11309,Secrecy in Clipper Chip\n\nThe serial number o...,11
11310,Hi !\n\nI am interested in the source of FEAL ...,11
11311,"The actual algorithm is classified, however, t...",11
11312,\n\tThis appears to be generic calling upon th...,11


In [22]:
df = df.replace(r'\n',' ', regex=True) 
df = df.replace(r'\t',' ', regex=True) 
df = df.replace(r"[^a-zA-Z]+",' ', regex=True) 
df['text'] = df['text'].str.lower()
df

,text,target
0,i was wondering if anyone out there could enli...,7
1,i recently posted an article asking what kind ...,7
2,it depends on your priorities a lot of people...,7
3,an excellent automatic can be found in the sub...,7
4,ford and his automobile i need information on...,7
...,...,...
11309,secrecy in clipper chip the serial number of t...,11
11310,hi i am interested in the source of feal encry...,11
11311,the actual algorithm is classified however the...,11
11312,this appears to be generic calling upon the n...,11


In [48]:
df = pd.read_csv("test.csv")
df

,text,target
0,do you have weitek s address phone number i d...,1
1,i certainly do use it whenever i have to do t...,1
2,hello i am looking to add voice input capabili...,1
3,australian pattern recognition society nd call...,1
4,hi noel i ve made some attempts to write a co...,1
...,...,...
1650,excerpts from netnews alt atheism apr re thoug...,0
1651,that s right humans have gone somewhat beyond...,0
1652,did you miss my post on this topic with the q...,0
1653,archive name atheism resources alt atheism arc...,0


In [49]:
df.replace("", float("NaN"), inplace=True)
df.dropna(inplace=True)
df.isnull().values.any()
df

,text,target
0,do you have weitek s address phone number i d...,1
1,i certainly do use it whenever i have to do t...,1
2,hello i am looking to add voice input capabili...,1
3,australian pattern recognition society nd call...,1
4,hi noel i ve made some attempts to write a co...,1
...,...,...
1650,excerpts from netnews alt atheism apr re thoug...,0
1651,that s right humans have gone somewhat beyond...,0
1652,did you miss my post on this topic with the q...,0
1653,archive name atheism resources alt atheism arc...,0


In [67]:
df = pd.read_csv("20_newsgroup.csv")
df.replace("", float("NaN"), inplace=True)
df.dropna(inplace=True)
df['text'] = df['text'].str.lower()
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('date', axis=1)
df = df.drop('title', axis=1)
df

,text,target
0,i was wondering if anyone out there could enli...,7
1,i recently posted an article asking what kind ...,7
2,\nit depends on your priorities. a lot of peo...,7
3,an excellent automatic can be found in the sub...,7
4,: ford and his automobile. i need information...,7
...,...,...
11309,secrecy in clipper chip\n\nthe serial number o...,11
11310,hi !\n\ni am interested in the source of feal ...,11
11311,"the actual algorithm is classified, however, t...",11
11312,\n\tthis appears to be generic calling upon th...,11


In [ ]:
df.to()

In [63]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=0.2, random_state=42)


In [64]:
max_len = 1000 
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>' # out of vocabulary token
vocab_size = 5000

In [65]:
tokenizer = Tokenizer(num_words = vocab_size, 
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(x_train)

In [66]:
word_index = tokenizer.word_index
total_words = len(word_index)
total_words

91901

In [53]:
training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)  
                                
testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

In [54]:
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

Shape of training tensor:  (1296, 1000)
Shape of testing tensor:  (324, 1000)


In [55]:
vocab_size = 5000
embedding_dim = 16
drop_value = 0.2
n_dense = 24
n_lstm = 128
drop_lstm = 0.2
model = Sequential()
model.add(Embedding(vocab_size,
                    embedding_dim,
                    input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(24, activation='relu'))
model.add(Dropout(drop_value))
model.add(Dense(1, activation='sigmoid'))

In [56]:
model2 = Sequential()
model2.add(Embedding(vocab_size,
                     embedding_dim,
                     input_length = max_len))
model2.add(Bidirectional(LSTM(n_lstm,
                              return_sequences = False)))
model2.add(Dropout(drop_lstm))
model2.add(Dense(1, activation='sigmoid'))

In [57]:
model2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1000, 16)          80000     
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              148480    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 228,737
Trainable params: 228,737
Non-trainable params: 0
_________________________________________________________________


In [58]:
model2.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics=['accuracy'])

In [ ]:
num_epochs = 10
early_stop = EarlyStopping(monitor = 'val_loss',
                           patience = 2)
history = model2.fit(training_padded,
                     y_train,
                     epochs = num_epochs,
                     validation_data = (testing_padded, y_test),
                     callbacks = [early_stop],
                     verbose = 2)